# Тренируем большую модель на анекдотах

In [25]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np

In [26]:
! wget https://raw.githubusercontent.com/rzabolotin/dl_zero_to_hero/refs/heads/main/anecdots.txt

--2024-11-11 18:52:01--  https://raw.githubusercontent.com/rzabolotin/dl_zero_to_hero/refs/heads/main/anecdots.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1578474 (1.5M) [text/plain]
Saving to: ‘anecdots.txt.1’

anecdots.txt.1      100%[===================>]   1.50M  --.-KB/s    in 0.07s   

2024-11-11 18:52:01 (23.1 MB/s) - ‘anecdots.txt.1’ saved [1578474/1578474]



In [27]:
with open("anecdots.txt", "r") as f_in:
    book = f_in.read()
print(book[:500])


??????????????????????????????? 

?О АВТОМАШИНАХ И ИХ ВОДИТЕЛЯХ ? 

??????????????????????????????? 

Несмотря на красный свет светофора, Луиза пересекла 

перекресток на своей машине и была остановлена полицейским. 

- Мадам, разве вы не видели красного света? - интересуется он. 

- Простите, мосье регулировщик, - отвечает она. - Красный 

свет я видела, а вот вас не заметила. 

Женщина-водитель после столкновения с другой машиной говорит: 

- Это моя вина! 

- Нет, мадам, - галантно отвечает 


# Словарь и токенайзер

In [28]:
vocab = sorted(list(set("".join(book))), key=lambda v: "\t" if v == "." else v)
vocab_size = len(vocab)

In [29]:
char_to_index = {char: index for index, char in enumerate(vocab)}
index_to_char = {index: char for char, index in char_to_index.items()}

def tokenize(char):
    return char_to_index.get(char, 0)

def untokenize(index):
    return index_to_char.get(index, " ")

# Готовим данные для обучения

In [30]:
data = torch.tensor([tokenize(x) for x in book], dtype=torch.long)
print(data, data.shape)

tensor([ 1, 29, 29,  ...,  3,  2,  1]) torch.Size([914848])


In [31]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [32]:
def get_batch(split, batch_size = 4):
    data = val_data if split == "valid" else train_data
    idx = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in idx])
    Y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    X, Y = X.to(device), Y.to(device)
    return(X,Y)

# Функции полезные

In [33]:
@torch.no_grad()
def evaluate_model(model):
    model.eval()
    scores = {}
    for split in ['train', 'valid']:
        loss = 0
        for i in range(n_eval):
            X, Y = get_batch(split, batch_size=batch_size)
            _, loss_i = model(X, Y)
            loss += loss_i.item()
        scores[split] = loss / n_eval
    model.train()
    return scores

# Константы

In [35]:
block_size = 256
batch_size = 64
dropout = 0.2
n_embd = 384
dropout = 0.2
n_layer = 6
n_head = 6
device = 'cuda'
n_iter = 5000
n_eval = 200
learning_rate = 2e-4

# Модель

In [36]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


In [37]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [38]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [39]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


In [40]:
class Model7(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


# Тренировка

In [41]:
model7 = Model7()
model7 = model7.to(device)
optimizer7 = torch.optim.AdamW(model7.parameters(), lr=learning_rate)

In [42]:
total_params = sum(p.numel() for p in model7.parameters())
print(f"The model has {total_params:,} trainable parameters.")

The model has 10,849,680 trainable parameters.


In [45]:
%%time
m = model7
o = optimizer7
for i in range(n_iter):
    X,Y = get_batch('train', batch_size=batch_size)
    logits, loss = m(X, Y)
    o.zero_grad(set_to_none=True)
    loss.backward()
    o.step()
    if i%1000 == 0:
        scores = evaluate_model(m)
        print(f"Loss train: {scores['train']:.4f}, valid {scores['valid']:.4f}")
print(loss.item())

Loss train: 2.5857, valid 2.6173
Loss train: 1.9065, valid 1.9895
Loss train: 1.5622, valid 1.7096
Loss train: 1.3873, valid 1.5975
Loss train: 1.2730, valid 1.5469
1.2663553953170776
CPU times: user 38min 54s, sys: 5min 47s, total: 44min 42s
Wall time: 44min 51s


In [46]:
max_tokens = 1000
prompt = torch.zeros([1, 1], dtype = torch.long)
prompt = prompt.to(device)
print("".join([untokenize(x) for x in m.generate(prompt, max_tokens).tolist()[0]]))

. 

Фрамилиционер: 

- Эй, почему видчик. После бокальное как-то у церх время 

родильности, и кап ни осталось мне успела и кричил? 

Двести дорогулочку к учительницу. Ленит на карман и говорит: 

- Чем это ты теперь это куда? 

Вон отвечает: 

- Еще бы нет, а что с всегда он скакой еще раз выкирой? 

Полсуй, воды и купил рей салез горию образил в деревню: 

- Мне картоке, вы мне попал ударитовился прилить на дорожениями моих молодая? 

- Ну улыба то. У дали битгрись на радио в Эйстере, а порогулый 


Бильорард... слушайся. 

Утрой, если в госте по гослуге замуж. Туни вон незамужем: 

- Сейчас, что славу ничего цепь измененного глохневающего 

замерева! 

- Тогда как вы что, делжен никудал? 

- Хоть что муж приходит? 

- А зачем?! А! 

Муж 

Муж (обидо): 

- А как вы вылезаиваете, чтобы некоторить предлить своих 

салобных вздохом сегоднямие. 

Солдат-лист предстадавил она кого-нибудь, а жена, а 

когда может, ничего не верниться. 

Трамвается, приходит хлодок, где же и брачно за мяж. 

In [47]:
%%time
m = model7
o = optimizer7
for i in range(n_iter):
    X,Y = get_batch('train', batch_size=batch_size)
    logits, loss = m(X, Y)
    o.zero_grad(set_to_none=True)
    loss.backward()
    o.step()
    if i%1000 == 0:
        scores = evaluate_model(m)
        print(f"Loss train: {scores['train']:.4f}, valid {scores['valid']:.4f}")
print(loss.item())

Loss train: 1.1861, valid 1.5327
Loss train: 1.1039, valid 1.5374
Loss train: 1.0300, valid 1.5488
Loss train: 0.9536, valid 1.5716
Loss train: 0.8851, valid 1.5996
1.0129832029342651
CPU times: user 38min 48s, sys: 5min 45s, total: 44min 33s
Wall time: 44min 46s


In [48]:
max_tokens = 1000
prompt = torch.zeros([1, 1], dtype = torch.long)
prompt = prompt.to(device)
print("".join([untokenize(x) for x in m.generate(prompt, max_tokens).tolist()[0]]))

... 

- Ну тогда возьмен только и ничего не очень мясо. 

Попробовал через волкрасвенный вечер двинеет и спрашивает: 

- Как с кем не на грязнее, и скоро сильное время опытные запротив 

подзаки. 

Он вернулся на автобусе обычной коп. Потрезв возвращение с 

опуговорить в кладочкам светом плохо, сначала кофе. 

Очень спутился из разушения, стали урал. О, просыпаются 

перед соседней Смер Капендел Бри жены в родсти у другого 

кофе: 

- Ребята, два. Проклять ночью. 

- Вот когда делаешь, ты меня еще долго, когда то медовый месяц 

туда сам увидишь за ними угол казанычения. 

- И думаете, если мы уже хоть такая уж слышать, что из них 

стояли... 

Могилен и кричит: 

- Вообще-то насильно? 

Да но вот, где анего. 

Бросайтский следование: 

- Догоги наступите мне кот счастью задачную присантитутку, что на похороны 

дали. Он копрелывает мужик, на-у-ши пепчетали "весе вешей". 

- Да не все-та одина мужика - ежик и следы. 

- Блина! 

У мужиков-то стоит деджурные по путылке. 

- Дгай мясо р

In [49]:
torch.save(model7, 'model7_full.pth')

In [50]:
model7new = torch.load('model7_full.pth')
model7new.to(device)
model7new.eval()

<ipython-input-50-abb3c2459eef>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model7new = torch.load('model7_full.pth')


Model7(
  (token_embedding_table): Embedding(144, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
   

In [51]:
max_tokens = 1000
prompt = torch.zeros([1, 1], dtype = torch.long)
prompt = prompt.to(device)
print("".join([untokenize(x) for x in model7new.generate(prompt, max_tokens).tolist()[0]]))

.А... 

- И я постанули. 

Турист: 

- А вот это до цыплянка по колге  живота? 

Ученица: 

- Семей выходи за ним в поле аистор, под рулем. 

Официант водит психиатру: 

- Я хочу подпить не могу на это старать радиаку выговаривать неосомнадцев, 

пока не заметил, что бы показать в своем слуга уюгом виски. 

- Он вчера в шитльнице говорит клубере. 

- Пожалуйста, - спрашивает официант у верность у клетки, - обиделся 

воскресенье нам не обратится к хмуровому сосед соседом. - Понял? 

- Да мы, где ввосполн! Вешали его на 26 лет. Последний дергает 

попугать в бутылку водки. 

- Ты чего будешь повезло, вышло замуж? - спрашивает официант. 

- Хорошо, что ты будешь великолепно. 

- Я спросил у своей жене, - заключенно сказал своему подруге: 

- У меня тебя были вычистыть свою дочь, я испытываю. 

- Вот испытываю шутиться за ти каждого круге. Это 

надоело доходаточно, такого же я сблява ее мужские сильнее. 

Врач опытанно несколько дней жилье, если ты не должно обрусить ни 

успевать себя к